# **Tutorial GROVER - DNA Language Model**

Melissa Sanabria, Jonas Hirsch, Anna R. Poetsch

Biomedical Genomics, Biotechnology Center, Center for Molecular and Cellular Bioengineering, Technische Universität Dresden  
melissa.sanabria@tu-dresden.de arpoetsch@gmail.com

**Acknowledgements**: Pierre Joubert for his feedback.

# Overview

This tutorial will show you how to use the DNA language model GROVER to perform fine-tuning tasks to investigate genome biology.

[GROVER](https://https://www.biorxiv.org/content/10.1101/2023.07.19.549677v1) ("Genome Rules Obtained Via Extracted Representations") is a foundation DNA language model with an optimized vocabulary for the human genome. It has been pre-trained on the human genome and needs to be trained a second time, or fine-tuned, to perform specific tasks.

For this tutorial we chose an end-to-end example to fine-tune the pre-tained GROVER to predict DNA binding by the CCCTC-Binding Factor (CTCF). The task is to recognize which sites that contain a CTCF binding motif are actually bound by the protein. We will be using ChIP-seq data from HepG2 cells obtained from ENCODE.  

The models and files can be obtained from the linked paths at Google Colab for convenience. In addition, the files are also on Zenodo to assure long-term availability.

Throughout this tutorial, you will find ***MODIFY*** signs that indicate pieces of code that are task specific, meaning that they are specific for CTCF binding site prediction, and you can modify them for your desired task.

**WARNING**: Even though Google Colab gives you access to a GPU, which is necessary for this computationally expensive work, the free version only gives you access to a very simple device and the training might take a while.

In the version of Colab, which is free of charge, notebooks can run for a maximum of 12 hours, depending on availability and your usage patterns.  

## Setup

First, save a copy of this notebook in your Google Drive by navigating to 'File' then 'Save a copy in Drive...'.

Once you've opened your own copy, make sure you have enabled the GPU runtime for Google Colab by navigating to the menu 'Runtime', select 'Change runtime type' and set the runtime to 'T4 GPU'.


## Install dependencies

In [ ]:
!pip install transformers
!pip install wget

## Import necessary packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import OneHotEncoder
import wget
import os
import gdown
import shutil

# Prepare Data

**MODIFY!**

This section is task dependent. We will explain how to get obtain CTCF motifs in the human genome and label which of these motifs have ChIP-Seq signal that indicates CTCF binding.

First, we will download the file:

In [ ]:
CTCF_motifs_with_peak_annotation_url = "https://drive.google.com/uc?export=download&id=1Or21JkcZTxAZJcwdlRPlw4mINjRHKjjY"
file_name = "CTCF_motifs_with_peak_annotation.tsv"
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly

print("Starting downloading")
file_name = wget.download(CTCF_motifs_with_peak_annotation_url, out=file_name)
print(file_name)

This file shows the results from the command:

```
bedtools intersect -loj
```

Columns 0 - 8 are from the CTCF motif file and columns 9 - 18 are from the peak file

In this file we have listed each CTCF motif in the genome which has been detected by the FIMO software. Columns 0-8 gives us information about the motif itself.

In columns 9-18 we describe the ChIP-Seq data for each motif. If a ChIP-Seq peak was found at this motif, the coordinates of this peak are indicated. If there was no peak, there will be a period in these columns.

You can take a look at the table below.

In [ ]:
annotations = pd.read_csv('CTCF_motifs_with_peak_annotation.tsv', sep='\t', header=None)
annotations

Let's make the table more readable for the algorithm.

We create a new column called "class" where we replace the "." by 0, where 0 is our negative class that indicates that there is no peak. And the rest is replaced by 1, where 1 is our positive class that indicates there is a peak.

In [ ]:
annotations["class"] = annotations[9].apply(lambda x: 1 if x != "." else 0) # 1 if there is a peak, 0 if there is no peak
annotations

Now, let's make a new table that only takes the data we will need. Which are the columns 0,3, and 4. These represent chromosome, start, end and our class (whether the motif had a ChIP-Seq peak or not)

In [ ]:
data = annotations[[0, 3, 4, "class"]]
data

Let's rename the columns to remember what each column means.

In [ ]:
data.columns = ["chr", "start", "end", "class"]
data.head()

There are some chromosomes with names like chr1_gl000191. These represent scaffolds that were not successfully placed within the human genome assemlby. Let's filter them out here and see how many motifs we've filtered out.

In [ ]:
len_before = len(data)
data = data.loc[data["chr"].apply(lambda x: True if len(x) <= 5 else False)]
data = data.sort_values(by=["chr"], inplace=False)
len_after = len(data)
print("Number of motifs filtered out: ", len_before - len_after)

A very important reminder when you work with large amount of data is to visualize it.

Let's compare how many CTCF motif sites we have with how many of them have a peak in the ChiPseq data.

There are around 85k CTCF motif sites in hg19 (our human reference genome), and around 30k of them have a peak in the ChIPseq data.

In [ ]:
nr_motifs = len(data)
nr_motifs_with_peak = len(data.loc[data["class"] == 1])

fig  = plt.figure()
plt.bar(x=["# motifs in hg19", "# motifs with peak"], height=[nr_motifs, nr_motifs_with_peak])
plt.show()

print("# of motifs in hg19: ", nr_motifs)
print("# of motifs with peak: ", nr_motifs_with_peak)

For this specific task, we calculate the center of the motif and then add 500 nucleotides up- and downstream. This is to capture the sequence context of the motif, which is what GROVER was trained to work with.

In [ ]:
center_of_motif = data["start"] + (data["end"] - data["start"])//2

# add 500 nucleotides up- and downstream (because we do it symmetrical we don't need strand information, otherwise be careful here)
data["start"] = center_of_motif - 500
data["end"] = center_of_motif + 500
data

# Create dataset

From now on, we come back to general instructions that apply for any fine-tuning task.

Please remember that to do this yourself you will need your data to look like what we've shown so far. That means you will need a data frame with four columns: chr, start, end, and class

So far, we have not mentioned any Language Model terminology. Now, we need to change our sequence from nucleotides to tokens (or words).

First, we need to retrieve the byte pair tokenized sequences from hg19.

**WARNING**: Depending on your internet connection, this might take a while.

In [ ]:
file_name = "tokenized_chromosomes.zip"
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly

print("Starting downloading")
gdown.download("https://drive.google.com/uc?export=download&id=1W_D7T_SGIiRgJZ2q9WR5l0leCmURgMSv", file_name, quiet=False)

print("Unzipping file")
shutil.unpack_archive(file_name, "tokenized_chromosomes")


The tokenized chromosomes are stored in tokenized_chromosomes/chr_#.pkl where # ranges from 1 to 24 where 23 = X and 24 = Y

In order to make your work easier, we created a mapper function. It maps nucleotide position to token position.

In [ ]:
def create_mapper(tokenized_chr):
    mapper = []
    for i in range(len(tokenized_chr)):
        for e in range(len(tokenized_chr[i])):
            mapper.append(i)
    return mapper

Then we process all our data to map nucleotide to token positions. This takes a little while.



In [ ]:
tokenized_seqs = []
for chrom in data["chr"].unique(): # get a list of all chromosome names
    chrom_nr = chrom[3:] # get the chromosome number
    if chrom_nr == "X":
        chrom_nr = 23
    elif chrom_nr == "Y":
        chrom_nr = 24
    # slice the data for the current chromosome
    data_chr = data.loc[data["chr"] == chrom]
    # load BP tokenized chromosome
    with open(f"tokenized_chromosomes/chr_{chrom_nr}.pkl", "rb") as f:
        tokenized_chr = pickle.load(f) # is a list of tokens

    # load mapper that maps nucleotide position to token position
    print(f"loading mapper {chrom}")
    mapper = create_mapper(tokenized_chr)

    tokenized_starts = data.loc[data["chr"] == chrom]["start"].apply(lambda x: mapper[x])
    tokenized_ends = data.loc[data["chr"] == chrom]["end"].apply(lambda x: mapper[x])
    for start, end in zip(tokenized_starts, tokenized_ends):
        tokenized_seqs.append(tokenized_chr[start:end])


grover_dataset = pd.DataFrame({'X': tokenized_seqs, 'class': data["class"]})

Let's take a look at our tokenized dataset

In [ ]:
grover_dataset

After getting our tokenized dataset, we need to split the samples into train, validation and test. We will go for the standard partitions, 80% for training, 10% for testing and 10% for validation.

In [ ]:
train_data = grover_dataset.sample(frac=0.8, random_state=42)
test_data = grover_dataset.drop(train_data.index)
val_data = test_data.sample(frac=0.5, random_state=42)
test_data = test_data.drop(val_data.index)

print("Total number of samples", len(grover_dataset))
print("Number of samples for training,", len(train_data))
print("Number of samples for testing,", len(test_data))
print("Number of samples for validation,", len(val_data))

# GROVER in action

First, we need to download the pre-trained GROVER model.

In [ ]:
file_name = "GROVER_pretrained.zip"
if os.path.exists(file_name):
  os.remove(file_name) # if exists, remove it directly

print("Starting downloading")
gdown.download("https://drive.google.com/uc?export=download&id=1fMwBkFx3uUUVvxNggQH3xcfYCBvz572Y", file_name, quiet=False)

print("Unzipping file")
shutil.unpack_archive(file_name, "GROVER_pretrained")

We use the BERT implementation from the transformer package to then create a GROVER tokenizer.

In [ ]:
grover_tokenizer = BertTokenizer.from_pretrained("GROVER_pretrained/")

In order to make your work easier, we create a class to process the Dataset created for Grover.

In [ ]:
class GroverDataSet(Dataset):
    def __init__(self, sequences, y, tokenizer, max_length=512):
        print("Loading GROVER Dataset")
        self.BERTtokenizer = tokenizer # to convert ATG ATCGA CG -> [CLS] ATG ATCGA CG [SEP] -> [2, 123, 456, 789, 101, 3]
        self.sequences = sequences
        self.max_length = max_length
        self.y = np.array(y, dtype=np.float32).reshape(-1, 1)
        self.label_encoder = OneHotEncoder(sparse_output=False)
        self.label_encoder.fit(self.y)
        self.y = self.label_encoder.transform(self.y)
        self.y = torch.tensor(self.y)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        tokenizer_res = self.BERTtokenizer.encode_plus(seq, add_special_tokens=True, padding="max_length", return_tensors="pt", max_length=self.max_length, truncation=True)
        ids = tokenizer_res["input_ids"].squeeze(0)
        attention_masks = tokenizer_res["attention_mask"]
        return ids, self.y[idx], attention_masks, idx



Let's create the data loaders for each set (train, test, validation), which are required to work with PyTorch libraries.

In [ ]:
train = GroverDataSet(train_data["X"].values, train_data["class"], grover_tokenizer, max_length=310)
val = GroverDataSet(val_data["X"].values, val_data["class"], grover_tokenizer, max_length=310)
test = GroverDataSet(test_data["X"].values, test_data["class"], grover_tokenizer, max_length=310)

train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader = DataLoader(val, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

## Train model

We have created the methods train_loop, val_loop and test_loop to make your work easier.

In the training loop, it performs the backpropagation and it outputs the progress every 10 batches.

In the validation loop, it first modifies the probability predictions using a threshold of 0.5. In other words, a sample is considered negative (class 0) if the probability is lower than 0.5 and a sample is considered positive (class 1) if the probability is higher than 0.5. Then the loss and metrics are computed.

**WARNING**: Even though Google Colab gives you access to a GPU, the free version gives you a very simple device. Then the training might take a while.

In the version of Colab that is free of charge notebooks can run for at most 12 hours, depending on availability and your usage patterns.

In [ ]:
def train_loop(model, train_loader, epoch, device, optimizer, criterion):
    model.train()
    for batch_idx, (data, target, attention_masks, _) in enumerate(train_loader):
        target = torch.tensor(target, dtype=torch.float32)
        data, target, attention_masks = data.to(device), target.to(device), attention_masks.to(device)
        optimizer.zero_grad()
        # Forward pass
        logits = model(data, attention_mask=attention_masks).logits
        # Calculate loss
        loss = criterion(logits, target)
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

def val_loop(model, val_loader, device, criterion):
    model.eval()
    val_loss = 0
    preds = []
    targets = []
    with torch.no_grad():
        for data, target, attention_masks, idx in val_loader:
            #target = torch.tensor(target, dtype=torch.float32)
            data, target, attention_masks = data.to(device), target.to(device), attention_masks.to(device)
            logits = model(data, attention_mask=attention_masks).logits
            val_loss += criterion(logits, target).item()

            probs = torch.sigmoid(logits).detach().cpu()
            # turn probs into predictions
            for probs_i in probs:
                for j in range(len(probs_i)):
                    probs_i[j] = 1 if probs_i[j] > 0.5 else 0
            label_ids = target.cpu()
            preds.append(probs)
            targets.append(label_ids)
        targets = torch.vstack(targets)
        preds = torch.vstack(preds)
        val_loss /= len(val_loader)
        print('\nValidation set: Average loss: {:.4f}\n'.format(val_loss))
        print(classification_report(targets, preds))
        return val_loss


For the training process we choose:

*   an Adam optimizer but you can explore the [optimizers](https://huggingface.co/transformers/v3.0.2/main_classes/optimizer_schedules.html) available in huggingface
*   learning rate of 0.000001 since for fine-tuning tasks it is better to have a very low learning rate
*   a total number of epochs of 4

A higher number of epochs is ideal but Colab limits the amount of time you can run some code. We advice you to try with your own GPU resources with at least 10 epochs.



In [ ]:
learning_rate = 0.000001
nr_epochs = 4

Below is where we actually fine-tune BERT for this task.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
grover = BertForSequenceClassification.from_pretrained("GROVER_pretrained/", num_labels=2).to(device)
optimizer = transformers.AdamW(grover.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

best_val_loss = val_loop(grover, val_loader, device, criterion)
for epoch in range(nr_epochs):
    print(f"Epoch {epoch}")
    train_loop(grover, train_loader, epoch, device, optimizer, criterion)
    val_loss = val_loop(grover, val_loader, device, criterion)
    # save the model every time the validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        grover_tokenizer.save_pretrained("GROVER_finetuned/")
        grover.save_pretrained("GROVER_finetuned/")
        print("Saved model")


## Test model

After training the model, we can see the performance of our model on the test set, which are samples that the model has not previously seen.

Here we will see some metrics such as accuracy, f1 score, precision and recall. You can explore the [sklearn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) to choose other options.

In [ ]:
def test_loop(model, test_loader, device):
    preds = []
    targets = []
    model.eval()
    with torch.no_grad():
        for data, target, attention_masks, idx in test_loader:
            data, target, attention_masks = data.to(device), target.to(device), attention_masks.to(device)
            logits = model(data, attention_mask=attention_masks).logits
            probs = torch.sigmoid(logits).detach().cpu()

            for probs_i in probs:
                for j in range(len(probs_i)):
                    probs_i[j] = 1 if probs_i[j] > 0.5 else 0
            label_ids = target.cpu()
            preds.append(probs)
            targets.append(label_ids)

        targets = torch.vstack(targets)
        preds = torch.vstack(preds)
        print(classification_report(targets, preds))

We first load our previously finetuned Grover model and then predict with the test set.

In [ ]:
grover = BertForSequenceClassification.from_pretrained('GROVER_finetuned/')
test_loop(grover, test_loader, device)

# Now, it's your turn

We use CTCF binding site as example but you can also use it for many other tasks such as promoter prediction, structural variants, integration sites of transposable elements, other binding sites, and many more.